## Import Libraries

In [1]:
from time import sleep

In [2]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [3]:
import matplotlib.pyplot as plt
plt.ion()

In [4]:
from pydgilib_extra import *

Set the path to `dgilib.dll`.

In [5]:
dgilib_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\dgilib.dll"

## Data Logging

Create a figure for the plot.

In [21]:
fig = plt.figure(figsize=(10, 6))
fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [7]:
config_dict = {
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "gpio_delay_time" : 0.0008,
    "fig": fig,
    "verbose": 0,
}

Perform the measurement.

In [8]:
data = []
fig.clf()

with DGILibExtra(dgilib_path, **config_dict) as dgilib:
    dgilib.device_reset()
    dgilib.logger_start()
    sleep(1)
    dgilib.update_callback()
    while not all(dgilib.data[INTERFACE_GPIO][1][-1]):
        dgilib.update_callback()
#         sleep(1)
    sleep(0.1)
    dgilib.logger_stop()
    data = dgilib.data

## Analysis

In [9]:
aes_charge, aes_time = power_and_time_per_pulse(data, 2)

In [10]:
flash_charge, flash_time = power_and_time_per_pulse(data, 3)

In [11]:
aes_encrypt_charge = aes_charge[0::2]
aes_decrypt_charge = aes_charge[1::2]
aes_encrypt_time = aes_time[0::2]
aes_decrypt_time = aes_time[1::2]

In [12]:
flash_write_charge = flash_charge[0::2]
flash_read_charge = flash_charge[1::2]
flash_write_time = flash_time[0::2]
flash_read_time = flash_time[1::2]

In [13]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 1
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(aes_encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)
print(f"MAX_AES_BLOCKS: {MIN_AES_BLOCKS + len(encrypt_charge) - 1}")

In [14]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [15]:
results = []
for y in [aes_encrypt_charge, aes_decrypt_charge, aes_encrypt_time, aes_decrypt_time, flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 10
    # variables        = 2
    chi-square         = 1.6384e-09
    reduced chi-square = 2.0480e-10
    Akaike info crit   = -221.321267
    Bayesian info crit = -220.716097
[[Variables]]
    slope:      3.2103e-05 +/- 9.8474e-08 (0.31%) (init = 0)
    intercept:  4.0605e-06 +/- 9.7762e-06 (240.76%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.886

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 10
    # variables        = 2
    chi-square         = 8.9580e-10
    reduced chi-square = 1.1197e-10
    Akaike info crit   = -227.358925
    Bayesian info crit = -226.753754
[[Variables]]
    slope:      3.3313e-05 +/- 7.2816e-08 (0.22%) (init = 0)
    intercept:  5.2947e-07 +/- 7.2289e-06 (1365.31%) (init = 1)
[[Correlations]] (unre

In [16]:
fig2 = plt.figure(figsize=(9, 8))

In [17]:
charge_color = 'r'
time_color = 'b'

In [18]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [mC]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [19]:
lines = []
lines += ax1.plot(num_bytes, aes_encrypt_charge, charge_color+'-', label='AES Encrypt Charge')
lines += ax1.plot(num_bytes, aes_decrypt_charge, charge_color+'--', label='AES Decrypt Charge')
lines += ax2.plot(num_bytes, aes_encrypt_time, time_color+'-', label='AES Encrypt Time')
lines += ax2.plot(num_bytes, aes_decrypt_time, time_color+'--', label='AES Decrypt Time')
lines += ax1.plot(num_bytes, flash_write_charge, charge_color+'-.', label='Flash Write Charge')
lines += ax1.plot(num_bytes, flash_read_charge, charge_color+':', label='Flash Read Charge')
lines += ax2.plot(num_bytes, flash_write_time, time_color+'-.', label='Flash Write Time')
lines += ax2.plot(num_bytes, flash_read_time, time_color+':', label='Flash Read Time')
ax1.legend(handles=lines)
# [aes_encrypt_charge, aes_decrypt_charge, aes_encrypt_time, aes_decrypt_time, flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
ax1.set_title(f"AES Encrypt Charge: Base {results[0].params['intercept'].value:.03} mC plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"AES Decrypt Charge: Base {results[1].params['intercept'].value:.03} mC plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"AES Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"AES Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n" +
             f"Flash Write Charge: Base {results[4].params['intercept'].value:.03} mC plus {results[4].params['slope'].value:.03} mC per byte\n" +
             f"Flash Read Charge: Base {results[5].params['intercept'].value:.03} mC plus {results[5].params['slope'].value:.03} mC per byte\n" +
             f"Flash Write Time: Base {results[6].params['intercept'].value:.03} s plus {results[6].params['slope'].value:.03} s per byte\n" +
             f"Flash Read Time: Base {results[7].params['intercept'].value:.03} s plus {results[7].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [20]:
# data